<a href="https://colab.research.google.com/github/pnkjjha/BuildNExperiment/blob/main/googlecolab_Spark_notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.3/317.3 MB 4.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.2-py2.py3-none-any.whl size=317812365 sha256=541b0f119648d31a67bae38caf51e4a30f32879ccbdfed74780c84934de75751
  Stored in directory: /root/.cache/pip/wheels/34/34/bd/03944534c44b677cd5859f248090daa9fb27b3c8f8e5f49574
Successfully built pyspark


In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as f

In [ ]:
import requests

In [ ]:
spark = SparkSession.builder.appName('practice').getOrCreate()

In [ ]:
url = "https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2024-01.parquet"
response = requests.get(url, stream=True)
with open("yellow_tripdata_2024-01.parquet", "wb") as f:
    for chunk in response.iter_content(chunk_size=1024):
        if chunk:
            f.write(chunk)



In [ ]:
# Read the Parquet file from the local file system
df1 = spark.read.parquet("yellow_tripdata_2024-01.parquet")

In [ ]:
df1.show()

+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+
|VendorID|tpep_pickup_datetime|tpep_dropoff_datetime|passenger_count|trip_distance|RatecodeID|store_and_fwd_flag|PULocationID|DOLocationID|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|congestion_surcharge|Airport_fee|
+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+
|       2| 2024-01-01 00:57:55|  2024-01-01 01:17:43|              1|         1.72|         1|                 N|         186|          79|           2|       17.7|  1.0|    0.5|       0.

In [ ]:
df1.printSchema()

root
 |-- VendorID: integer (nullable = true)
 |-- tpep_pickup_datetime: timestamp_ntz (nullable = true)
 |-- tpep_dropoff_datetime: timestamp_ntz (nullable = true)
 |-- passenger_count: long (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- RatecodeID: long (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- PULocationID: integer (nullable = true)
 |-- DOLocationID: integer (nullable = true)
 |-- payment_type: long (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- extra: double (nullable = true)
 |-- mta_tax: double (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- tolls_amount: double (nullable = true)
 |-- improvement_surcharge: double (nullable = true)
 |-- total_amount: double (nullable = true)
 |-- congestion_surcharge: double (nullable = true)
 |-- Airport_fee: double (nullable = true)



In [ ]:
df1.count()

2964624

In [ ]:
df1.first()

Row(VendorID=2, tpep_pickup_datetime=datetime.datetime(2024, 1, 1, 0, 57, 55), tpep_dropoff_datetime=datetime.datetime(2024, 1, 1, 1, 17, 43), passenger_count=1, trip_distance=1.72, RatecodeID=1, store_and_fwd_flag='N', PULocationID=186, DOLocationID=79, payment_type=2, fare_amount=17.7, extra=1.0, mta_tax=0.5, tip_amount=0.0, tolls_amount=0.0, improvement_surcharge=1.0, total_amount=22.7, congestion_surcharge=2.5, Airport_fee=0.0)

In [ ]:
df1.filter(df1.VendorID == 1).count()

729732

In [ ]:
df1.groupBy('VendorID').count().show()

+--------+-------+
|VendorID|  count|
+--------+-------+
|       1| 729732|
|       2|2234632|
|       6|    260|
+--------+-------+



when mentioning groupy you dont have to mention select because they are always same set of columns.

In [ ]:
df1.groupBy('VendorID').sum('tip_amount').orderBy('VendorID', ascending=False).show()

+--------+------------------+
|VendorID|   sum(tip_amount)|
+--------+------------------+
|       6|               0.0|
|       2| 7700953.170000889|
|       1|2188647.1400000826|
+--------+------------------+



In [ ]:
df1.show(5)

+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+
|VendorID|tpep_pickup_datetime|tpep_dropoff_datetime|passenger_count|trip_distance|RatecodeID|store_and_fwd_flag|PULocationID|DOLocationID|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|congestion_surcharge|Airport_fee|
+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+
|       2| 2024-01-01 00:57:55|  2024-01-01 01:17:43|              1|         1.72|         1|                 N|         186|          79|           2|       17.7|  1.0|    0.5|       0.

In [ ]:
df1.createOrReplaceTempView("taxi_data")

In [ ]:
%sql select vendorID, sum(tip_amount) from taxi_data group by VendorID

UsageError: Line magic function `%sql` not found.


In [ ]:
spark.sql("""select vendorID
, sum(tip_amount)
from taxi_data
group by VendorID""").show()

+--------+------------------+
|vendorID|   sum(tip_amount)|
+--------+------------------+
|       1|2188647.1400000826|
|       2| 7700953.170000889|
|       6|               0.0|
+--------+------------------+



In [ ]:
arr = [[[2,3,4],'first row'],[[11,22,33],'second row'],[[111,222,333],'third row']]
error_df=spark.createDataFrame(arr,schema=["arr","row desc"])


In [ ]:
error_df.show()

+---------------+----------+
|            arr|  row desc|
+---------------+----------+
|      [2, 3, 4]| first row|
|   [11, 22, 33]|second row|
|[111, 222, 333]| third row|
+---------------+----------+



In [ ]:
error_df.select(f.explode('arr').alias('arr1_name'),"row desc").show()

+---------+----------+
|arr1_name|  row desc|
+---------+----------+
|        2| first row|
|        3| first row|
|        4| first row|
|       11|second row|
|       22|second row|
|       33|second row|
|      111| third row|
|      222| third row|
|      333| third row|
+---------+----------+



explode might be useful in dealing with JSON dataset where a column can have a array of value. and we want it to normalize.

In [ ]:
error_df.cache()

DataFrame[arr: array<bigint>, row desc: string]

reading csv data in pyspark dataframe
https://docs.databricks.com/en/ingestion/file-upload/download-internet-files.html
```csv_df=spark.read.csv('https://raw.githubusercontent.com/thewiremonkey/factbook.csv/master/data/c2001.csv',header=True,inferSchema=True)```
<br>
```csv_df.show()```

In [ ]:
team_df = [["A",2,3,4]
           ,["B",1,2,3]
           ,["C",8,9,10]]
col = ["Name","forward","center","back"]
df2 = spark.createDataFrame(team_df,col)

In [ ]:
df2.show()

+----+-------+------+----+
|Name|forward|center|back|
+----+-------+------+----+
|   A|      2|     3|   4|
|   B|      1|     2|   3|
|   C|      8|     9|  10|
+----+-------+------+----+



In [ ]:
df3 = df2.unpivot(ids=["Name"]\
         ,values = ["forward","center","back"]\
         ,variableColumnName="position"\
         ,valueColumnName="Count")

In [ ]:
df3.show()

+----+--------+-----+
|Name|position|Count|
+----+--------+-----+
|   A| forward|    2|
|   A|  center|    3|
|   A|    back|    4|
|   B| forward|    1|
|   B|  center|    2|
|   B|    back|    3|
|   C| forward|    8|
|   C|  center|    9|
|   C|    back|   10|
+----+--------+-----+



In [ ]:
df3.offset(3).show()

+----+--------+-----+
|Name|position|Count|
+----+--------+-----+
|   B| forward|    1|
|   B|  center|    2|
|   B|    back|    3|
|   C| forward|    8|
|   C|  center|    9|
|   C|    back|   10|
+----+--------+-----+



In [41]:
def getmysubstring(str_length,str_lit):
  adj_dict={}
  for chr_index in range(len(str_lit)-1):

    if str_lit[chr_index] not in adj_dict.keys():
      adj_dict[str_lit[chr_index]]=[]

    if str_lit[chr_index] != str_lit[chr_index+1] and \
       str_lit[chr_index+1] not in adj_dict[str_lit[chr_index]]:

      adj_dict[str_lit[chr_index]]= adj_dict[str_lit[chr_index]] + [str_lit[chr_index+1]]

    if chr_index > 0 and \
       str_lit[chr_index] != str_lit[chr_index-1] and \
       str_lit[chr_index-1] not in adj_dict[str_lit[chr_index]]:
      adj_dict[str_lit[chr_index]]= adj_dict[str_lit[chr_index]] + [str_lit[chr_index-1]]
  key='b'
  vstr=''
  print(adj_dict)
  for i in range(str_length):
    if len(adj_dict[key]) >= 2 and\
        key not in vstr:
      vstr = vstr + key
      key = adj_dict[key][0]

  if len(vstr) < str_length:
    return 'not possible'
  else:
    return vstr

getmysubstring(4,"abaddfg")

{'a': ['b', 'd'], 'b': ['a'], 'd': ['a', 'f'], 'f': ['g', 'd']}


'not possible'